# 1. Logistic Regression

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility (나중에도 같은 결과 제공 - seed 부여)
torch.manual_seed(1)

## Training Data

Consider the following classification problem: given the number of hours each student spent watching the lecture and working in the code lab, predict whether the student passed or failed a course.

For example, the first (index 0) student watched the lecture for 1 hour and spent 2 hours in the lab session ([1, 2]), and ended up failing the course ([0])

In [7]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]] # 6 by 2
y_data = [[0], [0], [0], [1], [1], [1]] # 6 by 1

As always, we need these data to be in `torch.Tensor` format, so we convert them.

In [9]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [10]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


## Computing the Hypothesis

$$ H(X) = \frac{1}{1 + e^{-W^T X}}$$

PyTorch has a `torch.exp()` function that resembles the exponential function.

In [11]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


We can use it to compute the hypothesis function conveniently.

In [12]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [13]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [14]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


## Computing the Cost Function

$$ cost(W) = -\frac{1}{m} \sum{y \log{(H(x)) + (1-y) (\log{(1 - H(x))})}} $$

We want to measure the difference between `hypothesis` and `y_train`.

In [15]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


The loss can be computed as follows:

In [17]:
losses = -(y_train * torch.log(hypothesis) + 
            (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


Then, we just `.mean()` to take the mean of these individual losses.

In [18]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [19]:
# binary cross entropy 바로 구하는 함수
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

## Whole Training Procedure

In [20]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1) # SGD 사용

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번 마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d} / {} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0 / 1000 Cost: 0.693147
Epoch  100 / 1000 Cost: 0.134722
Epoch  200 / 1000 Cost: 0.080643
Epoch  300 / 1000 Cost: 0.057900
Epoch  400 / 1000 Cost: 0.045300
Epoch  500 / 1000 Cost: 0.037261
Epoch  600 / 1000 Cost: 0.031672
Epoch  700 / 1000 Cost: 0.027556
Epoch  800 / 1000 Cost: 0.024394
Epoch  900 / 1000 Cost: 0.021888
Epoch 1000 / 1000 Cost: 0.019852


## Evaluation
After we finish training the model, we want to check how well our model fits the training set.

In [22]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b) # 원래는 evaluation 시 testset 사용
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward0>)


We can change **hypothesis** (real number from 0 to 1) to **binary predictions** (either 0 or 1) by comparing them to 0.5

In [26]:
prediction = hypothesis >= torch.FloatTensor([0.5]) # 0.5보다 크면 1(true), 작거나 같으면 0(false)
print(prediction.type())
print(prediction[:5])

torch.BoolTensor
tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [30]:
# 잘 예측했는지 확인
print(prediction[:5])
print(y_train[:5])

correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])
tensor([[True],
        [True],
        [True],
        [True],
        [True]])


# 2. Higher Implementation with Class

In [49]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1) # 2개 입력받을 것
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [54]:
model = BinaryClassifier()

In [55]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 20 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d} / {} Cost: {:.6f} Accuracy: {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0 / 100 Cost: 0.799465 Accuracy: 50.00%
Epoch   20 / 100 Cost: 0.614591 Accuracy: 83.33%
Epoch   40 / 100 Cost: 0.421928 Accuracy: 83.33%
Epoch   60 / 100 Cost: 0.262911 Accuracy: 83.33%
Epoch   80 / 100 Cost: 0.163836 Accuracy: 100.00%
Epoch  100 / 100 Cost: 0.137029 Accuracy: 100.00%
